# Covid model
This project is for WAMM 2021 done by Hinda Nguyen and Katie Johnston

In [1]:
#import libraries that might be useful
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.integrate import odeint

In [2]:
#define some variables that will be useful
total_pop = 1e6 
people = [total_pop - 1, 1, 0]

people_interact = 10
infect_chance = 0.5
days_infectious = 3 


In [3]:
def one_day(people):
    
    infect_today = 0
    for i in range(int(people[1])):
        for n in range(int(people_interact)):
            rand = random.random()
            if( rand < people[0]/total_pop): 
                if random.random() < infect_chance:
                    people[0]-=1
                    people[1]+=1
                    infect_today += 1 
    return (people, infect_today)


In [4]:
people = [total_pop - 1, 1, 0]
people_infected=[]
def fourteen_days(people):
    for n in range(14):
        one_day(people)
        people_infected.append(people[1])
        if(people_infected[n] == total_pop): 
            return people_infected
    return people_infected

people_infected = fourteen_days(people)

        

In [ ]:
people = [total_pop - 1, 1, 0]
people_infected_ideal = []#=np.zeros((14))
def fourteen_days_ideal(people):#runs 14 days of simulation and stores number of infected people in an array
    for n in range(14):
        one_day_ideal(people)
        people_infected_ideal.append(people[1])
        if(people_infected_ideal[n] > total_pop): #if everyone gets infected stop
            return people_infected_ideal
    return people_infected_ideal
# print(fourteen_days(people))
people_infected_ideal = fourteen_days(people)
print(people_infected)

In [ ]:
#ideas for shifting array and then adding new value
people_recovered=np.zeros((days_infectious))

#want number of people infected each day
people_infected_copy= np.zeros(np.array(people_infected).shape)
people_infected_copy[0] = people_infected[0]
people_infected_copy[1:] = np.array(people_infected[1:])- np.array(people_infected[:-1])

# print(people_infected[:-1])
# print(people_infected[1:])
# print(people_infected_copy)
for n in range(len(people_infected)): 
    people_recovered=np.append(people_recovered,people_infected_copy[0]) #adds the first value in people_infected to people_recovered
    people_infected_copy=np.roll(people_infected_copy,-1) #shifts everything in people_infected left
    people_infected_copy[-1]=0 #zero is where the new value would be
print(people_infected_copy)
print(people_recovered)
print(people_infected)

In [ ]:
#ideas for combining fourteen_days and people_recovered


def fourteen_days(people = [total_pop - 1, 1, 0]):
    people_all = []
    people_all.append(people.copy())
    people_infected=np.zeros((days_infectious))
    people_infected[-2] = people[1]

    for n in range(14):
        (people, infect_today) = one_day(people)
        
        people_infected[-1] = infect_today
        
        #move people from I to R
        people[2] += people_infected[0]
        people[1] -= people_infected[0]
        people_infected = np.roll(people_infected,-1)
        people_infected[-1]=0
        
        people_all.append(people.copy())
        
    return np.array(people_all)

people_all = fourteen_days()


In [ ]:
#graph
plt.plot(people_all[:, 0], label = "S")
plt.plot(people_all[:, 1], label = "I")
plt.plot(people_all[:, 2], label = "R")
plt.legend()
plt.xlabel("Time (days)")
plt.ylabel("People (millions)")

In [ ]:
plt.plot(people_infected, label = "infected")
plt.plot(people_infected_ideal, label = "infected ideal")
plt.plot(total_pop - np.array(people_infected), label = "suspetible")
plt.legend()
plt.plot(people_recovered)

In [ ]:
def one_day_ideal(people):
    #Run 1 day of simulation
    #people will interact with people_interact number of people and then may or may not be infected
    infected = people[1]
    sus = people[0]
    people[0]-=people_interact*infect_chance*infected*sus/total_pop
    people[1]+=people_interact*infect_chance*infected*sus/total_pop
    #could change people array as needed for new number of infections, ... or make new variable
    return people

#what are some other functions that might be useful for our simulation

In [ ]:
#taken from https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/

# Total population, N.
N = total_pop
t_max = 10
# Initial number of infected and recovered individuals, I0 and R0.
I0, R0 = 1, 0
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0
# Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
beta = people_interact*infect_chance#people_interact/total_pop
print(beta)
gamma = 1./days_infectious
# A grid of time points (in days)
t = np.arange(0,t_max, 1)

# The SIR model differential equations.
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

# Initial conditions vector
y0 = S0, I0, R0
# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T


# Plot the data on three separate curves for S(t), I(t) and R(t)
fig = plt.figure()
plt.plot(t, S/1000, 'b', alpha=0.5, lw=2, label='Susceptible')
plt.plot(t, I/1000, 'r', alpha=0.5, lw=2, label='Infected')
plt.plot(t, R/1000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
plt.xlabel('Time (days)')
plt.ylabel('Number (1000s)')
plt.legend()
plt.show()